# Xente Solution Using Deep Neural Nets
 By Atwine Mugume
 
I have been working on an online classification problem where we have to detect fraud.
Since I am now alittle conversant with some of these methods I want to try my luck at using them so that I can see if this will improve my score on the leader boards

In [1]:
#let me import some of the libraries that I will be using
import pandas as pd
import numpy as np

In [2]:
#let's read the data into the notebook
train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

In [4]:
#the data that I have imported above is not yet preprocessed
train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [5]:
#I have already preprocessed this data into two different csv files
#The only thing I need here is the id from the test data so that I can make a submission
test.head(4)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4


In [7]:
#let me take out the transaction ID
TransactionID = test['TransactionId']
TransactionID[:5] #let's see the top 5 ids

0    TransactionId_50600
1    TransactionId_95109
2    TransactionId_47357
3    TransactionId_28185
4    TransactionId_22140
Name: TransactionId, dtype: object

### Pre-processed data

I am not going to bring in the pre-processed data on which I am going to run my neural net

In [8]:
train_prc = pd.read_csv('Preprocessed_data/train_prc.csv')
test_prc = pd.read_csv('Preprocessed_data/test_prc.csv')

In [10]:
train_prc.shape, test.shape

((95662, 92), (45019, 15))

In [11]:
train_prc.columns

Index(['Unnamed: 0', 'BatchId_0', 'BatchId_1', 'BatchId_2', 'BatchId_3',
       'BatchId_4', 'BatchId_5', 'BatchId_6', 'BatchId_7', 'BatchId_8',
       'BatchId_9', 'BatchId_10', 'BatchId_11', 'BatchId_12', 'BatchId_13',
       'BatchId_14', 'BatchId_15', 'BatchId_16', 'BatchId_17', 'AccountId_0',
       'AccountId_1', 'AccountId_2', 'AccountId_3', 'AccountId_4',
       'AccountId_5', 'AccountId_6', 'AccountId_7', 'AccountId_8',
       'AccountId_9', 'AccountId_10', 'AccountId_11', 'AccountId_12',
       'SubscriptionId_0', 'SubscriptionId_1', 'SubscriptionId_2',
       'SubscriptionId_3', 'SubscriptionId_4', 'SubscriptionId_5',
       'SubscriptionId_6', 'SubscriptionId_7', 'SubscriptionId_8',
       'SubscriptionId_9', 'SubscriptionId_10', 'SubscriptionId_11',
       'SubscriptionId_12', 'CustomerId_0', 'CustomerId_1', 'CustomerId_2',
       'CustomerId_3', 'CustomerId_4', 'CustomerId_5', 'CustomerId_6',
       'CustomerId_7', 'CustomerId_8', 'CustomerId_9', 'CustomerId_10',
       '

In [12]:
#let's take out the label of the data so that it doesn't get lost in there somewhere
label = train_prc['FraudResult']

#then we also drop this field from the data
train_prc = train_prc.drop(columns='FraudResult')

### Validation Set

In [18]:
#I need to keep a validation set of the data before we go so far
#I am going to keep the last 15000
x_val = train_prc.loc[80662:,]
y_val = label[80662:]

### Useable Data

In [23]:
train_df = train_prc.loc[:80661,]
label_ = label[:80662]

In [24]:
train_df.shape, label_.shape

((80662, 91), (80662,))

### Now let us split the data

We are going to use the sklearn model of splitting to plit the data and the we are going to work with the data imbalance before we beging to introduce our neural entwork.


In [25]:
from sklearn.model_selection import train_test_split

#let's split the data now
x_train, x_test, y_train, y_test = train_test_split(train_df,label_,test_size=0.2,random_state=2019)

In [27]:
#let's see the shapes
x_train.shape, y_train.shape,  x_test.shape, y_test.shape

((64529, 91), (64529,), (16133, 91), (16133,))

## Data Imbalance

Most times in fraud related datasets we normally have less fraud than there are fraud cases. We generally have data imbalance and would like to sort that out before we put the data into the  ML tuner.

In [28]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(replacement=False)
x_train_subsample, y_train_subsample = rus.fit_sample(x_train, y_train)
print(x_train.shape)
print(x_train_subsample.shape)
print(np.bincount(y_train_subsample))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(64529, 91)
(214, 91)
[107 107]


## Keras Model Building

Now that we have dealt with some of these things in the data its time to begin to deal with it by building a Deep Neural Network to Help solve this problem.

I am going to be using Google Colab because they have a free GPU and TPU
